In [5]:
import os
import pathlib
import numpy as np
import tensorflow as tf

In [6]:
# Define the directory of the dataset
data_dir = pathlib.Path('/Users/filippouslenghi/msa/dataset')
# Collects the path of all the files within the dataset
data_paths = [str(path) for path in list(data_dir.glob("*/*.jpg"))]
print(f"Images in the dataset: {len(data_paths)}")

Images in the dataset: 24951


In [7]:
# Create the training and test sets using the paths of the images
from sklearn.model_selection import train_test_split
train_paths, test_paths = train_test_split(data_paths, test_size=0.2, shuffle=True)

In [8]:
# Create the respective tf.data.Dataset objects
train = tf.data.Dataset.from_tensor_slices(train_paths)
test = tf.data.Dataset.from_tensor_slices(test_paths)

Metal device set to: Apple M1 Pro


2022-07-04 19:46:49.081918: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-04 19:46:49.082099: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
# Get the class names
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

['Cats' 'Dogs']


In [10]:
# Create a validation set
val_size = int(len(list(train)) * 0.2)
train = train.skip(val_size)
val = train.take(val_size)

In [11]:
# Set initial params for the loader
batch_size = 64
img_height = 200
img_width = 200

In [12]:
def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    one_hot = parts[-2] == class_names
    # Integer encode the label
    return tf.argmax(one_hot)

In [14]:
def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    try:
        img = tf.io.decode_jpeg(img, channels=3)
    except:
        img = tf.io.decode_bmp(img, channels=3)
    # Resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

In [15]:
def process_path(file_path):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [16]:
# Create a dataset of image, label pairs
train = train.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val = val.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
test = test.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

In [17]:
# Configure dataset for performance
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

train = configure_for_performance(train)
val = configure_for_performance(val)
test = configure_for_performance(test)

In [18]:
num_classes = 2

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

In [19]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [21]:
model.fit(
    train,
    validation_data=val,
    epochs=10
)

Epoch 1/10
250/250 [==============================] - 26s 102ms/step - loss: 0.4237 - accuracy: 0.8038 - val_loss: 0.3946 - val_accuracy: 0.8186
Epoch 2/10
250/250 [==============================] - 25s 100ms/step - loss: 0.3772 - accuracy: 0.8271 - val_loss: 0.3390 - val_accuracy: 0.8540
Epoch 3/10
250/250 [==============================] - 25s 98ms/step - loss: 0.3312 - accuracy: 0.8546 - val_loss: 0.3456 - val_accuracy: 0.8467
Epoch 4/10
250/250 [==============================] - 27s 109ms/step - loss: 0.2714 - accuracy: 0.8856 - val_loss: 0.2521 - val_accuracy: 0.8935
Epoch 5/10
250/250 [==============================] - 26s 103ms/step - loss: 0.2226 - accuracy: 0.9074 - val_loss: 0.2526 - val_accuracy: 0.8930
Epoch 6/10
250/250 [==============================] - 26s 101ms/step - loss: 0.1723 - accuracy: 0.9294 - val_loss: 0.1622 - val_accuracy: 0.9341
Epoch 7/10
250/250 [==============================] - 25s 98ms/step - loss: 0.1312 - accuracy: 0.9502 - val_loss: 0.1536 - val_accu